# Deconvolutions

Demonstration of deconvolutions.

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

Init Plugin
Init Graph Optimizer
Init Kernel


## Example 1: (4,4) -> (2,2) and back

We can implement the convolution by flattening the kernel over the positions on the input. This yields a matrix of shape (4,16) in the example. We then also have to flatten the input to get a shape of (16,1). The dot product then has the shapes (4,16) x (16,1) = (4,1).
Finally, the output has to be reshaped into shape (2,2).


In [24]:
# Example 1.1
#
# Transposing the matrix explicitly
#

x = np.ones([1,4,4,1])
x = x.reshape(16,1) # Flatten input to shape (16,1)

# Flatten kernel on input. Stride (1,1) yields 4 positions.
kernel = np.asarray([[1,1,1],[1,1,1],[1,1,1]])
C = np.asarray([[1,1,1,0,1,1,1,0,1,1,1,0,0,0,0,0],
                [0,1,1,1,0,1,1,1,0,1,1,1,0,0,0,0],
                [0,0,0,0,1,1,1,0,1,1,1,0,1,1,1,0],
                [0,0,0,0,0,1,1,1,0,1,1,1,0,1,1,1]
                ])

## =========================================================
# Forward / convolution
y = np.dot(C, x)
print("Output shape forward pass/convolution: ", y.shape)

## =========================================================
# Backward / deconvolution, transposed convolution
C_t = C.transpose()
x = np.asarray([[9,9],[9,9]])
x = x.reshape(4,1) # Flatten input

y = np.dot(C_t, x)

print("Output shape backward pass/deconvolution: ", y.shape)

Output shape forward pass/convolution:  (4, 1)
Output shape backward pass/deconvolution:  (16, 1)


We can implement the same in tensorflow with `Conv2D` and `Conv2DTranspose`. We can set the kernel weights to 1 to replicate the example from above.

Note that we need to provide a channel dimension and we also provide a batch dimension here.

Looking at the values of the outputs we can see that they are the same as above. In the forward case the result is 

```
[[9,9],
[9,9]]
```

and in the backward case it is

```
[[9,18,18,9],
[18,36,36,18],
[18,36,36,18],
[9,18,18,9]]

In [18]:
# Example 1.2
#
# Convolution

x = np.ones([1,4,4,1])
x = tf.convert_to_tensor(x, dtype=tf.float32)
x = tf.reshape(x,[1,4,4,1])

# Model
model = tf.keras.Sequential([
    layers.Conv2D(1, (3,3), strides=(1,1), padding='valid', kernel_initializer='ones', use_bias=False)])

print("Input shape: ", x.shape)
y = model(x)
tf.print("Output shape: ", y.shape)
#print(y) # Uncomment to view the output values

Input shape:  (1, 4, 4, 1)
Output shape:  TensorShape([1, 2, 2, 1])


In [21]:
# Example 1.3
#
# Deconvolution / Transposed convolution

x = np.asarray([[9,9],[9,9]])
x = tf.convert_to_tensor(x, dtype=tf.float32)
x = tf.reshape(x,[1,2,2,1])

model = tf.keras.Sequential([
    layers.Conv2DTranspose(1,(3,3),strides=(1,1), padding='valid', kernel_initializer='ones', use_bias=False)
])

print("Input shape: ", x.shape)
y = model(x)
tf.print("Output shape: ", y.shape)
#print(y) # Uncomment to view the output values

Input shape:  (1, 2, 2, 1)
Output shape:  TensorShape([1, 4, 4, 1])


We can also implement the deconvolution with upsampling and a convolution.

In [30]:
# Example 1
# Deconvolution implemented with a convolution and padding. By using the conv layer the resizing can still be learned, despite the padding (upsampling).

x = np.asarray([[9,9],[9,9]])
x = tf.convert_to_tensor(x, dtype=tf.float32)
x = tf.reshape(x,[1,2,2,1])
print("Input shape: ", x.shape)

# Pad input with 0s
paddings = tf.constant([[0,0],[2,2],[2,2],[0,0]])
x = tf.pad(x, paddings=paddings)

model = tf.keras.Sequential([
    layers.Conv2D(1, (3,3), strides=(1,1), padding='valid', kernel_initializer='ones', use_bias=False)
])

y = model(x)
tf.print("Output shape: ", y.shape)
#print(y) # Uncomment to view the output values

Input shape:  (1, 2, 2, 1)
Output shape:  TensorShape([1, 4, 4, 1])


## Example 2: (5,5) -> (2,2) and back

In [40]:
# Example 2
# Stride (2,2) in input
x = tf.ones([1,5,5,1])

model = tf.keras.Sequential([
    layers.Conv2D(1,(3,3), strides=(2,2), padding='valid')
])

print("Input shape: ", x.shape)
y = model(x)
tf.print("Output shape: ", y.shape)


Input shape:  (1, 5, 5, 1)
Output shape:  TensorShape([1, 2, 2, 1])


In [46]:
# TODO

# Example 2
# Fractionally strided convolution
x = tf.ones([1,2,2,1])

# Pad input
paddings = tf.constant([[0,0], [2,2], [2,2],[0,0]])
x = tf.pad(x, paddings=paddings)


model = tf.keras.Sequential([
    layers.Conv2DTranspose(1, (3,3), strides=(1,1), padding='valid')
])

print("Input shape: ", x.shape)
y = model(x)
tf.print("Output shape: ", y.shape)

Input shape:  (1, 6, 6, 1)
Output shape:  TensorShape([1, 8, 8, 1])


In [ ]:
# Transposed convolution with a convolutional layer